In [19]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [20]:
import re
from nltk.tokenize import sent_tokenize
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset  # For built-in dataset

In [21]:
# Step 1: Load the GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()  # Set the model to evaluation mode

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [23]:
# Ensure pad token is defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [24]:
# Step 2: Load a built-in dataset
def load_and_sample_dataset():
    """
    Load and sample data from a built-in dataset for text generation.
    Returns:
        str: A sample prompt from the dataset.
    """
    dataset = load_dataset("ag_news", split="train")  # News dataset for conversational context
    sample = dataset.shuffle(seed=42).select(range(1))  # Select a random sample
    return sample[0]["text"]

In [25]:
# Step 3: Preprocess text data
def preprocess_text(text):
    """
    Clean and tokenize text input.
    """
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove non-alphanumeric characters
    sentences = sent_tokenize(text)
    return sentences

In [26]:
# Step 4: Generate conversational responses
def generate_response(prompt, max_length=150, temperature=0.7, top_k=50, top_p=0.9, repetition_penalty=1.2):
    """
    Generate a conversational response using GPT-2.
    """
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        do_sample=True,
        early_stopping=True,
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


In [27]:
# Step 5: Conversational system
def conversational_system():
    """
    Runs the multi-turn conversational system with built-in dataset support.
    """
    print("Conversational AI System (GPT-2)")
    print("Type 'exit' to end the conversation.\n")

    # Load a sample prompt from a dataset
    dataset_prompt = load_and_sample_dataset()
    print(f"Sample prompt loaded from dataset: {dataset_prompt}\n")

    conversation_context = f"Dataset Prompt: {dataset_prompt}\n"

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Conversation ended. Goodbye!")
            break

        # Append user input to context
        conversation_context += f"User: {user_input}\nBot: "

        # Generate bot response
        bot_response = generate_response(conversation_context)
        print(f"Bot: {bot_response}")

        # Append bot response to context
        conversation_context += bot_response + "\n"


In [32]:
# Step 6: Main program
if __name__ == "__main__":
    conversational_system()

Conversational AI System (GPT-2)
Type 'exit' to end the conversation.

Sample prompt loaded from dataset: Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.

You: tell me more about bangladesh
Bot: Dataset Prompt: Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.
User: tell me more about bangladesh
Bot: พ้-ɐtʒnĪmār·́ (Bengal) [ edit ]

"Bangalore". "Sri Lanka" is an English word for India or Sri Lankan landmass which means 'land of abundance'. In this context, it was used as if Bangalore were on Indian soil with its huge population being estimated around 11 billion according TOI's figures from 2014."A large part...of our nation comes out here because
You: exit
Conversation ended. Goodbye!
